In [6]:
import os
import sqlite3

# --- Helper to remove leading empty element if present ---
def clean_line(line):
    return line[1:] if line and line[0] == "" else line

###############################################################################
# 1) CREATE/KEEP THE SINGLE movement_data TABLE
###############################################################################
def init_db(db_path="actionPlus.sqlite"):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    create_sql = """
    CREATE TABLE IF NOT EXISTS movement_data (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      filename TEXT UNIQUE,
      participant_name TEXT,
      test_date TEXT,
      movement_type TEXT,
      foot_contact_frame INTEGER,
      release_frame INTEGER,

      [Arm_Abduction@Footplant] REAL,
      [Max_Abduction] REAL,
      [Shoulder_Angle@Footplant] REAL,
      [Max_ER] REAL,
      [Arm_Velo] REAL,
      [Max_Torso_Rot_Velo] REAL,
      [Torso_Angle@Footplant] REAL,

      [Score] REAL  -- <--- new column for our computed score
    );
    """
    c.execute(create_sql)
    conn.commit()
    conn.close()


###############################################################################
# 2) PARSE participant_name, test_date, movement_type from c3d path
###############################################################################
def parse_file_info(full_path):
    path = full_path.replace("\\", "/")
    parts = path.split("/")
    # Remove any empty strings at the start
    parts = [p for p in parts if p.strip() != ""]
    
    participant_name = "UnknownName"
    test_date = "UnknownDate"
    movement_type = "Unknown"

    if len(parts) >= 3:
        participant_name = parts[-3].replace("_KA", "").replace("_", " ").strip()
    if len(parts) >= 2:
        test_date = parts[-2].rstrip("_")

    fn_lower = parts[-1].lower()
    if "fastball" in fn_lower or "pitch" in fn_lower:
        movement_type = "Pitch"
    elif "shortstop" in fn_lower:
        movement_type = "Shortstop"
    elif "catchers" in fn_lower:
        movement_type = "Catchers"
    elif "crow" in fn_lower and "hop" in fn_lower:
        movement_type = "Crow Hop"

    return participant_name, test_date, movement_type


###############################################################################
# 3) PARSE aPlus_events.txt => foot_contact_frame, release_frame
#    (Process in chunks of 3; if the last chunk is partial, use what’s available)
###############################################################################
def parse_events_from_aPlus(events_path, capture_rate=300):
    if not os.path.isfile(events_path):
        print(f"❌ Missing events file: {events_path}")
        return {}
    with open(events_path, "r", encoding="utf-8") as f:
        lines = [clean_line(line.rstrip("\n").split("\t")) for line in f]
    if len(lines) < 2:
        print(f"⚠️ Not enough lines in {events_path}")
        return {}
    filenames_line = lines[0]
    # Find the first numeric data row (assumed to be after header rows)
    data_line = None
    for row in lines[2:]:
        if row and row[0].isdigit():
            data_line = row
            break
    if not data_line:
        print(f"⚠️ Could not find numeric data in {events_path}")
        return {}
    # If the first cell is a row index, skip it:
    data_vals = data_line[1:] if data_line[0].isdigit() else data_line
    events_dict = {}
    use_len = min(len(filenames_line), len(data_vals))
    col_idx = 0
    while col_idx < use_len:
        fn = filenames_line[col_idx].strip()
        # For each chunk, attempt to get up to 3 values:
        foot_str = data_vals[col_idx].strip() if col_idx < use_len else ""
        max_er_str = data_vals[col_idx+1].strip() if (col_idx+1) < use_len else ""
        rel_str = data_vals[col_idx+2].strip() if (col_idx+2) < use_len else ""
        try:
            fc = float(foot_str) if foot_str != "" else None
        except:
            fc = None
        try:
            rel = float(rel_str) if rel_str != "" else None
        except:
            rel = None
        fc_fr = int(round(fc * capture_rate)) if fc is not None else None
        rel_fr = int(round(rel * capture_rate)) if rel is not None else None
        events_dict[fn] = {
            "foot_contact_frame": fc_fr,
            "release_frame": rel_fr
        }
        col_idx += 3
    return events_dict


###############################################################################
# 4) PARSE APlusData.txt => kinematics in chunks of 6 columns
###############################################################################
def parse_aplus_kinematics(txt_path):
    if not os.path.isfile(txt_path):
        print(f"❌ APlusData file not found: {txt_path}")
        return []
    with open(txt_path, "r", encoding="utf-8") as f:
        lines = [clean_line(line.rstrip("\n").split("\t")) for line in f]
    if len(lines) < 2:
        print(f"⚠️ Not enough lines in {txt_path}")
        return []
    filenames_line = lines[0]
    varnames_line  = lines[1]
    # Find the first numeric data row
    data_line = None
    for row in lines[2:]:
        if row and row[0].isdigit():
            data_line = row
            break
    if not data_line:
        print(f"⚠️ Could not find numeric row in {txt_path}")
        return []

    # If the first cell is a row index, skip it:
    data_vals = data_line[1:] if data_line[0].isdigit() else data_line

    # We now have 7 columns per trial: 
    #   Arm_Abduction@Footplant, Max_Abduction, Shoulder_Angle@Footplant,
    #   Max_ER, Arm_Velo, Max_Torso_Rot_Velo, Torso_Angle@Footplant
    chunk_size = 7

    use_len = min(len(filenames_line), len(varnames_line), len(data_vals))
    n_trials = use_len // chunk_size

    results = []
    for t in range(n_trials):
        start = t * chunk_size
        end = start + chunk_size
        sub_vars = varnames_line[start:end]
        sub_vals = data_vals[start:end]
        if not sub_vars or not sub_vals:
            continue
        trial_dict = {}
        for i in range(len(sub_vars)):
            var_name = sub_vars[i].strip()
            val_str  = sub_vals[i].strip()
            try:
                val = float(val_str)
            except:
                val = None
            trial_dict[var_name] = val
        
        # Use the corresponding filename
        fn = filenames_line[start].strip() if start < len(filenames_line) else f"Unknown_File_{t}"
        trial_dict["filename"] = fn

        # Only append if we have a valid filename & some data
        if fn != "" and any(trial_dict.get(k) is not None for k in trial_dict if k != "filename"):
            results.append(trial_dict)
    return results

def compute_score(
    arm_velo: float,
    torso_velo: float,
    abd_footplant: float,
    shoulder_fp: float,
    max_er: float
) -> float:
    """
    Example scoring logic:
      1) arm_velo * 0.005
      2) torso_velo * 0.02
      3) abduction@footplant => -1 * (abd_footplant * 2)
      4) shoulder ER@footplant => piecewise logic
      5) max_er => piecewise logic => +10 if in 180..210, 0 if in 211..220,
         0 if in 179..(just below 180?), and -10 if <180 or >220
    """

    # Safety for None => 0
    arm_velo      = arm_velo      or 0
    torso_velo    = torso_velo    or 0
    abd_footplant = abd_footplant or 0
    shoulder_fp   = shoulder_fp   or 0
    max_er        = max_er        or 0

    score = 0.0

    # 1) Arm velocity
    score += arm_velo * 0.005

    # 2) Torso velocity
    score += torso_velo * 0.02

    # 3) Abduction @ footplant (negative factor)
    score += -1.0 * (abd_footplant * 2)

    # 4) Shoulder ER @ footplant piecewise
    if 35 <= shoulder_fp <= 75:
        score += 30
    elif 76 <= shoulder_fp <= 85:
        score += 15
    elif 86 <= shoulder_fp <= 95:
        score += 0
    elif 96 <= shoulder_fp <= 105:
        score -= 10
    elif shoulder_fp >= 106:
        score -= 20
    elif 25 <= shoulder_fp <= 34:
        score += 15
    elif 15 <= shoulder_fp <= 24:
        score += 0
    elif 5 <= shoulder_fp <= 14:
        score -= 10
    elif shoulder_fp < 5:
        score -= 20

    # 5) Max_ER piecewise
    er_score = 0
    if 180 <= max_er <= 210:
        er_score = 10
    elif 211 <= max_er <= 220:
        er_score = 0
    elif max_er > 220:
        er_score = -10
    elif max_er < 180:
        er_score = -10
    
    score += er_score

    return score


###############################################################################
# 5) INGEST DATA INTO movement_data TABLE (INSERT OR REPLACE so DB builds over time)
###############################################################################
def ingest_data(aPlusDataPath, aPlusEventsPath, db_path="actionPlus.sqlite"):
    events_dict = parse_events_from_aPlus(aPlusEventsPath, capture_rate=300)
    kinematics  = parse_aplus_kinematics(aPlusDataPath)
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    insert_sql = """
    INSERT OR REPLACE INTO movement_data (
      filename, participant_name, test_date, movement_type,
      foot_contact_frame, release_frame,

      [Arm_Abduction@Footplant],
      [Max_Abduction],
      [Shoulder_Angle@Footplant],
      [Max_ER],
      [Arm_Velo],
      [Max_Torso_Rot_Velo],
      [Torso_Angle@Footplant],

      [Score]
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    for row in kinematics:
        fn = row.get("filename", "").strip()
        if not fn:
            continue
        fc = events_dict.get(fn, {}).get("foot_contact_frame")
        rel= events_dict.get(fn, {}).get("release_frame")
        p_name, p_date, m_type = parse_file_info(fn)

        # Pull the numeric fields from row
        abd_fp    = row.get("Arm_Abduction@Footplant") or 0
        max_abd   = row.get("Max_Abduction") or 0
        shld_fp   = row.get("Shoulder_Angle@Footplant") or 0
        max_er    = row.get("Max_ER") or 0
        arm_velo  = row.get("Arm_Velo") or 0
        torso_velo= row.get("Max_Torso_Rot_Velo") or 0
        torso_ang = row.get("Torso_Angle@Footplant") or 0

        # Compute the score
        score_val = compute_score(
            arm_velo,
            torso_velo,
            abd_fp,
            shld_fp,
            max_er
        )

        vals = [
            fn,
            p_name,
            p_date,
            m_type,
            fc,
            rel,

            abd_fp,
            max_abd,
            shld_fp,
            max_er,
            arm_velo,
            torso_velo,
            torso_ang,

            score_val  # <--- store it
        ]
        c.execute(insert_sql, vals)

    conn.commit()
    conn.close()


###############################################################################
# 6) Example main – adjust file paths as needed
###############################################################################
if __name__ == "__main__":
    DB_PATH = "actionPlus.sqlite"
    APLUS_EVENTS_PATH = r"D:\Youth Pitch Design\Exports\aPlus_events.txt"
    APLUS_DATA_PATH = r"D:\Youth Pitch Design\Exports\APlusData.txt"
    init_db(DB_PATH)
    ingest_data(APLUS_DATA_PATH, APLUS_EVENTS_PATH, DB_PATH)
    print("✅ Done. Check movement_data in actionPlus.sqlite.")

################################################################################
# 7) Create report
################################################################################

import os, shutil
import sqlite3
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.platypus import Table, TableStyle

####################################################
# CONFIG
####################################################
DB_PATH = "actionPlus.sqlite"
OUTPUT_DIR = r"D:\Youth Pitch Design\Reports\Action+"
OUTPUT_DIR_TWO = r"G:\My Drive\Youth Pitch Reports\Reports\Action+"
LOGO_PATH = r"C:\Users\q\PycharmProjects\Youth Pitch Design\8ctane - Faded 8 to Blue.png"

# Text files for velocities:
AP_TORSO_V_FILE = r"D:\Youth Pitch Design\Exports\aPlus_torsoVelo.txt"
AP_ARM_V_FILE   = r"D:\Youth Pitch Design\Exports\aPlus_armVelo.txt"

# Image paths
ASSETS_DIR = r"C:\Users\q\PycharmProjects\Youth Pitch Design\assets"
IMG_FRONT_FP  = os.path.join(ASSETS_DIR, "Front@FP.png")
IMG_SAG_FP    = os.path.join(ASSETS_DIR, "sag@FP.png")
IMG_SAG_MAXER = os.path.join(ASSETS_DIR, "sag@MaxER.png")
IMG_SAG_REL   = os.path.join(ASSETS_DIR, "sag@Rel.png")

####################################################
# 1) DB / DATA HELPERS
####################################################
def get_report_data():
    conn = sqlite3.connect(DB_PATH)
    df_last = pd.read_sql_query(
        """
        SELECT participant_name, test_date
        FROM movement_data
        ORDER BY id DESC
        LIMIT 1
        """,
        conn
    )
    if df_last.empty:
        conn.close()
        raise ValueError("No data found in DB!")

    participant_name = df_last.iloc[0]["participant_name"]
    test_date        = df_last.iloc[0]["test_date"]

    # Now also get the average Score:
    query = f"""
    SELECT movement_type,
           AVG([Arm_Abduction@Footplant])     AS avg_abd,
           AVG([Max_Abduction])               AS avg_max_abd,
           AVG([Shoulder_Angle@Footplant])    AS avg_shoulder_fp,
           AVG([Max_ER])                      AS avg_max_er,
           AVG([Arm_Velo])                    AS avg_arm_velo,
           AVG([Max_Torso_Rot_Velo])          AS avg_torso_velo,
           AVG([Torso_Angle@Footplant])       AS avg_torso_angle,
           AVG([Score])                       AS avg_score   -- <--- new
    FROM movement_data
    WHERE participant_name = '{participant_name}'
      AND test_date        = '{test_date}'
    GROUP BY movement_type
    """
    df = pd.read_sql_query(query, conn)
    conn.close()

    # Sort by movement_type in a custom order
    order = ["Pitch", "Shortstop", "Catchers", "Crow Hop"]
    df["order"] = df["movement_type"].apply(lambda x: order.index(x) if x in order else 99)
    df = df.sort_values("order").drop(columns="order")
    return participant_name, test_date, df

def build_velocity_figure_all_frames(
    torso_txt,
    arm_txt,
    foot_contact_row,
    release_row
):
    """
    Plots the entire trial from frame=0 to frame=N-1 for both torso & arm velocity txt files.
    We skip top 7 lines, then treat columns>=2 as data.

    The x-axis is frames from 0..(N−1).
    We draw vertical gold lines at foot_contact_row-7 and release_row-7 if they're in range.

    This version sets the figure width=1600 & height=600 to make it wider and a bit taller.
    """
    # 1) Read TORSO (skip top 7 lines)
    df_torso = pd.read_csv(torso_txt, header=None, sep="\t").iloc[7:,:].copy()
    torso_data = df_torso.iloc[:, 2:].apply(pd.to_numeric, errors="coerce").abs()

    # 2) Read ARM
    df_arm = pd.read_csv(arm_txt, header=None, sep="\t").iloc[7:,:].copy()
    arm_data = df_arm.iloc[:, 2:].apply(pd.to_numeric, errors="coerce").abs()

    # 3) Number of total frames
    total_torso = len(torso_data)
    total_arm   = len(arm_data)
    total_rows  = max(total_torso, total_arm)

    x = np.arange(total_rows)

    def clamp_to_range(n, high):
        return max(0, min(n, high-1))

    if foot_contact_row is None:
        foot_contact_row = -9999
    if release_row is None:
        release_row = -9999

    foot_line = clamp_to_range(foot_contact_row - 7, total_rows)
    release_line = clamp_to_range(release_row - 7, total_rows)

    # 4) Build figure
    fig = go.Figure()
    fig.update_layout(
        width=1600,   # Widen the figure
        height=600,
        template="plotly_dark",
        title="Angular Velocities (Torso & Arm) – Full Trial Frames",
    )

    # TORSO => unify in one legend
    torso_cols = torso_data.columns
    for i, col_i in enumerate(torso_cols):
        y = torso_data[col_i].values
        show_legend = (i == 0)
        fig.add_trace(go.Scatter(
            x=x[:len(y)],
            y=y,
            mode="lines",
            name="Torso" if show_legend else None,
            line=dict(color="#d62728"),
            showlegend=show_legend,
            opacity=0.8
        ))

    # ARM => unify in one legend
    arm_cols = arm_data.columns
    for j, col_j in enumerate(arm_cols):
        y = arm_data[col_j].values
        show_legend = (j == 0)
        fig.add_trace(go.Scatter(
            x=x[:len(y)],
            y=y,
            mode="lines",
            name="Arm" if show_legend else None,
            line=dict(color="#2c99d4"),
            showlegend=show_legend,
            opacity=0.8
        ))

    combined = pd.concat([torso_data, arm_data], axis=1)
    y_min = float(np.nanmin(combined.values)) if not combined.empty else 0
    y_max = float(np.nanmax(combined.values)) if not combined.empty else 1

    # foot_contact line
    if 0 <= foot_line < total_rows:
        fig.add_shape(
            type="line",
            x0=foot_line,
            x1=foot_line,
            y0=y_min,
            y1=y_max,
            line=dict(color="gold", dash="dot", width=2)
        )

    # release line
    if 0 <= release_line < total_rows:
        fig.add_shape(
            type="line",
            x0=release_line,
            x1=release_line,
            y0=y_min,
            y1=y_max,
            line=dict(color="gold", dash="dash", width=3)
        )

    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom", y=1.02,
            xanchor="left",   x=0.05
        ),
        xaxis_title="Frames (0..end of trial)",
        yaxis_title="Velocity (absolute)"
    )
    return fig


####################################################
# 2) PDF GENERATION
####################################################
def generate_movement_report():
    # --- (A) RETRIEVE DB FRAMES ---
    conn = sqlite3.connect(DB_PATH)
    df_frames = pd.read_sql_query(
        """
        SELECT foot_contact_frame, release_frame
        FROM movement_data
        ORDER BY id DESC
        LIMIT 1
        """,
        conn
    )
    conn.close()

    if df_frames.empty:
        foot_contact_row = 0
        release_row      = 0
    else:
        foot_contact_row = df_frames.iloc[0]["foot_contact_frame"]
        release_row      = df_frames.iloc[0]["release_frame"]
        if pd.isna(foot_contact_row):
            foot_contact_row = 0
        if pd.isna(release_row):
            release_row = 0

    # --- (B) GET SUMMARY TABLE DATA ---
    participant_name, test_date, summary_df = get_report_data()

    # --- (C) BUILD & SAVE VELOCITY FIG ---
    fig_velo = build_velocity_figure_all_frames(
        AP_TORSO_V_FILE,
        AP_ARM_V_FILE,
        foot_contact_row,
        release_row
    )
    velo_png = "angular_velocity.png"
    fig_velo.write_image(velo_png)

    # --- (D) CREATE PDF ---
    page_width, page_height = 2000, 3200
    pdf_filename = os.path.join(OUTPUT_DIR, f"{participant_name} {test_date} Pitching Report.pdf")
    c = canvas.Canvas(pdf_filename, pagesize=(page_width, page_height))

    # BLACK BG
    c.setFillColorRGB(0,0,0)
    c.rect(0, 0, page_width, page_height, fill=1, stroke=0)

    brand_border = colors.HexColor("#4887a8")
    card_bg      = colors.HexColor("#1f1f1f")
    text_color   = colors.HexColor("#ffffff")

    # --- HEADER ---
    c.setFont("Helvetica-BoldOblique", 50)
    c.setFillColor(text_color)
    c.drawString(30, page_height - 60, "Movement Analysis Dashboard")

    c.setFont("Helvetica-Oblique", 34)
    c.drawString(30, page_height - 120, f"Athlete: {participant_name}")
    c.drawString(30, page_height - 170, f"Date: {test_date}")

    c.setStrokeColor(brand_border)
    c.setLineWidth(3)
    c.line(20, page_height - 185, page_width - 20, page_height - 185)

    # LOGO
    if os.path.exists(LOGO_PATH):
        c.drawImage(
            LOGO_PATH,
            page_width - 500,
            page_height - 180,
            width=398.06,
            height=160.03,
            preserveAspectRatio=True,
            mask='auto'
        )

    # --- TABLE OF AVERAGES ---
    table_df = summary_df.copy()
    # If your summary_df has a column 'avg_score' that you do NOT want in the table,
    # you can drop it. Or just rename everything except that column:
    table_df = table_df.rename(columns={
        "avg_abd":         "Abd @ FP",
        "avg_max_abd":     "Max Abd",
        "avg_shoulder_fp": "Arm Timing",
        "avg_max_er":      "Max ER",
        "avg_arm_velo":    "Arm Velo",
        "avg_torso_velo":  "Torso Velo",
        "avg_torso_angle": "Torso Ang@FP"
        # Note: if "avg_score" is in df, we won't rename it, or we can drop it:
        # "avg_score": "Score"  # or just not rename if we don't want it in the table
    })

    table_df = table_df.set_index("movement_type")
    desired_order = [mt for mt in ["Pitch", "Shortstop", "Catchers", "Crow Hop"] if mt in table_df.index]
    table_df = table_df.loc[desired_order]

    # Build list-of-lists for the table
    header_row = ["Movement Type"] + list(table_df.columns)
    table_data = [header_row]
    for mt, row in table_df.iterrows():
        row_vals = [mt]
        for col in table_df.columns:
            val = row[col]
            row_vals.append(f"{val:.1f}" if pd.notna(val) else "")
        table_data.append(row_vals)

    # Table coords & sizes
    table_card_x = 20
    table_card_y = page_height - 520
    table_card_w = page_width - 480
    table_card_h = 300

    # Draw the "card" for the table
    c.setFillColor(card_bg)
    c.setStrokeColor(brand_border)
    c.roundRect(table_card_x, table_card_y, table_card_w, table_card_h, 10, fill=1)

    # Table title
    c.setFillColor(text_color)
    c.setFont("Helvetica-BoldOblique", 34)
    c.drawString(table_card_x + 15, table_card_y + table_card_h - 40, "Movement Averages")

    # Build the actual table using ReportLab's Table
    from reportlab.platypus import Table, TableStyle
    t = Table(table_data, colWidths=[280] + [150]*(len(header_row)-1), rowHeights=48)
    t.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#333333")),
        ('TEXTCOLOR',  (0, 0), (-1, 0), text_color),
        ('FONTNAME',   (0, 0), (-1, 0), "Helvetica-Bold"),
        ('FONTSIZE',   (0, 0), (-1, 0), 20),
        ('ALIGN',      (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN',     (0, 0), (-1, -1), 'MIDDLE'),
        ('GRID',       (0, 0), (-1, -1), 1, colors.HexColor("#444444")),
        ('BACKGROUND', (0, 1), (-1, -1), colors.black),
        ('TEXTCOLOR',  (0, 1), (-1, -1), text_color),
        ('FONTNAME',   (0, 1), (-1, -1), "Helvetica"),
        ('FONTSIZE',   (0, 1), (-1, -1), 26),
    ]))
    tw, th = t.wrapOn(c, table_card_w-20, table_card_h-20)
    t.drawOn(c, table_card_x+25, table_card_y + table_card_h - 60 - th)

    ############################################################################
    # (1) Compute the final "stability_score" from summary_df["avg_score"]
    ############################################################################
    if "avg_score" in summary_df.columns and not summary_df.empty:
        stability_score = summary_df["avg_score"].mean()
    else:
        stability_score = 0.0

    ############################################################################
    # (2) Draw Score Box to the RIGHT of the table
    # So let's place it relative to table_card_x, table_card_y, etc.
    ############################################################################
    score_w = 340
    score_h = 300
    # place it 20 px to the right, and top-aligned
    score_x = table_card_x + table_card_w + 20
    score_y = table_card_y + (table_card_h - score_h)

    c.setFillColor(card_bg)
    c.setStrokeColor(brand_border)
    c.roundRect(score_x, score_y, score_w, score_h, 10, fill=1)

    c.setFillColor(text_color)

    c.setFont("Helvetica-Oblique", 24)
    c.drawString(score_x + 10, score_y + 20, "Higher = Better")

    c.setFont("Helvetica-BoldOblique", 40)
    c.drawString(score_x + 10, score_y + score_h - 40, "Kinematic Score")

    c.setFont("Helvetica-BoldOblique", 90)
    c.setFillColor(colors.HexColor("#32CD32"))  # lime color
    c.drawString(score_x + 40, score_y + 100, f"{stability_score:.1f}")

    ##########################################################
    # ANGULAR VELOCITIES TEXT + GRAPH IN ONE BOX
    ##########################################################
    def draw_wrapped_text(canvas_obj, text, x, y, max_width, font_name="Helvetica", font_size=24, leading=5):
        words = text.split()
        lines = []
        current_line = ""
        for word in words:
            test_line = f"{current_line} {word}".strip()
            if canvas_obj.stringWidth(test_line, font_name, font_size) <= max_width:
                current_line = test_line
            else:
                lines.append(current_line)
                current_line = word
        if current_line:
            lines.append(current_line)
        
        for i, line in enumerate(lines):
            canvas_obj.drawString(x, y - i*(font_size + leading), line)

    # We place the velocity box below the table, as usual
    top_text_graph_card_w = page_width - 100
    top_text_graph_card_h = 800
    top_text_graph_card_x = 50
    top_text_graph_card_y = table_card_y - top_text_graph_card_h - 30  # same small gap

    c.setFillColor(card_bg)
    c.setStrokeColor(brand_border)
    c.roundRect(top_text_graph_card_x, top_text_graph_card_y,
                top_text_graph_card_w, top_text_graph_card_h,
                10, fill=1)

    c.setFillColor(text_color)
    # Title
    c.setFont("Helvetica-Bold", 36)
    c.drawString(top_text_graph_card_x + 30, top_text_graph_card_y + top_text_graph_card_h - 50, 
                 "Angular Velocities")

    # Body text
    arm_text = (
        "Angular velocities are how fast the designated segment rotates around the proximal segment "
        "throughout the pitching motion. The kinematic sequence refers to sequential velocity peaks "
        "in the pelvis, torso, upper arm, forearm, and hand. Higher velocities in the trunk and arm "
        "have been linked to increased performance.\n\n"
        "This chart displays the entire trial's data (frame 0 through the final frame), with vertical "
        "gold lines at the foot contact and release frames if they exist."
    )

    c.setFont("Helvetica", 24)
    text_left  = top_text_graph_card_x + 30
    text_top   = top_text_graph_card_y + top_text_graph_card_h - 100
    max_width  = top_text_graph_card_w - 30
    draw_wrapped_text(c, arm_text, text_left, text_top, max_width, font_size=24)

    graph_img_y = top_text_graph_card_y + 50
    graph_img_h = 550
    graph_side_margin = 10
    graph_img_w = top_text_graph_card_w - (graph_side_margin * 2)

    c.drawImage(
        velo_png,
        text_left, 
        graph_img_y,  
        width=graph_img_w,
        height=graph_img_h,
        preserveAspectRatio=True,
        mask='auto'
    )

    # Move the next sections up...
    current_y = top_text_graph_card_y - 30

    def draw_text_image_block(title_str, body_str, image_paths=None, box_height=700):
        nonlocal current_y
        card_w = page_width - 100
        card_h = box_height
        card_x = 50
        card_y = current_y - card_h

        c.setFillColor(card_bg)
        c.setStrokeColor(brand_border)
        c.roundRect(card_x, card_y, card_w, card_h, 10, fill=1)

        c.setFillColor(text_color)
        c.setFont("Helvetica-Bold", 36)
        c.drawString(card_x + 30, card_y + card_h - 50, title_str)

        c.setFont("Helvetica", 24)
        text_left = card_x + 30
        text_top  = card_y + card_h - 100
        max_text_width = card_w - 60
        draw_wrapped_text(c, body_str, text_left, text_top, max_width=max_text_width, font_size=24)

        # Move images further down
        img_y_offset = 20  
        img_y = card_y + img_y_offset
        img_w = 650
        img_h = 500
        if image_paths:
            if len(image_paths) == 1:
                # center
                img_x = card_x + (card_w - img_w)/2
                c.drawImage(image_paths[0], img_x, img_y,
                            width=img_w, height=img_h,
                            preserveAspectRatio=True, mask='auto')
            elif len(image_paths) == 2:
                spacing = (card_w - 2*img_w) / 3
                left_img_x  = card_x + spacing
                right_img_x = card_x + spacing*2 + img_w
                c.drawImage(image_paths[0], left_img_x, img_y,
                            width=img_w, height=img_h,
                            preserveAspectRatio=True, mask='auto')
                c.drawImage(image_paths[1], right_img_x, img_y,
                            width=img_w, height=img_h,
                            preserveAspectRatio=True, mask='auto')

        current_y = card_y - 60
        return current_y

    # Horizontal Abduction
    ha_title = "Horizontal Abduction"
    ha_text  = (
        "Horizontal abduction is how far behind the body the arm/elbow gets during the pitching motion. "
        "Commonly referred to as the 'loading' of the arm, horizontal abduction has been linked to both velocity "
        "and arm health.\n\n"
        "Front@FP.png      Sag@FP.png"
    )
    draw_text_image_block(ha_title, ha_text, [IMG_FRONT_FP, IMG_SAG_FP], box_height=720)

    # Shoulder External Rotation
    ser_title = "Shoulder External Rotation"
    ser_text  = (
        "Shoulder external rotation is measured at both footplant and as a max value during the pitching motion. "
        "Shoulder external rotation at footplant is often referred to as 'arm timing.' An on-time arm is between "
        "33 and 77 degrees. Anything lower than 33 is deemed late; above 77 is deemed early.\n\n"
        "Max External rotation (often called layback) is how much the arm externally rotates during "
        "the pitching motion. A higher max ER has been linked to both arm health and velocity.\n\n"
        "Sag@MaxER.png      sag@Rel.png"
    )
    draw_text_image_block(ser_title, ser_text, [IMG_SAG_MAXER, IMG_SAG_REL], box_height=720)

    c.showPage()
    c.save()
    print(f"✅ PDF saved to: {pdf_filename}")

    for extra_dir in (OUTPUT_DIR_TWO,):
        os.makedirs(extra_dir, exist_ok=True)
        shutil.copy2(pdf_filename, os.path.join(extra_dir, os.path.basename(pdf_filename)))
    
if __name__ == "__main__":
    generate_movement_report()



✅ Done. Check movement_data in actionPlus.sqlite.
✅ PDF saved to: D:\Youth Pitch Design\Reports\Action+\Davis Gentle DG 2025-09-18 Pitching Report.pdf


In [ ]:
# import os
# import sqlite3
# 
# # --- Helper to remove leading empty element if present ---
# def clean_line(line):
#     return line[1:] if line and line[0] == "" else line
# 
# ###############################################################################
# # 1) CREATE/KEEP THE SINGLE movement_data TABLE
# ###############################################################################
# def init_db(db_path="actionPlus.sqlite"):
#     conn = sqlite3.connect(db_path)
#     c = conn.cursor()
#     create_sql = """
#     CREATE TABLE IF NOT EXISTS movement_data (
#       id INTEGER PRIMARY KEY AUTOINCREMENT,
#       filename TEXT UNIQUE,
#       participant_name TEXT,
#       test_date TEXT,
#       movement_type TEXT,
#       foot_contact_frame INTEGER,
#       release_frame INTEGER,
# 
#       [Arm_Abduction@Footplant] REAL,
#       [Max_Abduction] REAL,
#       [Shoulder_Angle@Footplant] REAL,
#       [Max_ER] REAL,
#       [Arm_Velo] REAL,
#       [Max_Torso_Rot_Velo] REAL,
#       [Torso_Angle@Footplant] REAL,
# 
#       [Score] REAL  -- <--- new column for our computed score
#     );
#     """
#     c.execute(create_sql)
#     conn.commit()
#     conn.close()
# 
# 
# ###############################################################################
# # 2) PARSE participant_name, test_date, movement_type from c3d path
# ###############################################################################
# def parse_file_info(full_path):
#     path = full_path.replace("\\", "/")
#     parts = path.split("/")
#     # Remove any empty strings at the start
#     parts = [p for p in parts if p.strip() != ""]
#     
#     participant_name = "UnknownName"
#     test_date = "UnknownDate"
#     movement_type = "Unknown"
# 
#     if len(parts) >= 3:
#         participant_name = parts[-3].replace("_KA", "").replace("_", " ").strip()
#     if len(parts) >= 2:
#         test_date = parts[-2].rstrip("_")
# 
#     fn_lower = parts[-1].lower()
#     if "fastball" in fn_lower or "pitch" in fn_lower:
#         movement_type = "Pitch"
#     elif "shortstop" in fn_lower:
#         movement_type = "Shortstop"
#     elif "catchers" in fn_lower:
#         movement_type = "Catchers"
#     elif "crow" in fn_lower and "hop" in fn_lower:
#         movement_type = "Crow Hop"
# 
#     return participant_name, test_date, movement_type
# 
# 
# ###############################################################################
# # 3) PARSE aPlus_events.txt => foot_contact_frame, release_frame
# #    (Process in chunks of 3; if the last chunk is partial, use what’s available)
# ###############################################################################
# def parse_events_from_aPlus(events_path, capture_rate=300):
#     if not os.path.isfile(events_path):
#         print(f"❌ Missing events file: {events_path}")
#         return {}
#     with open(events_path, "r", encoding="utf-8") as f:
#         lines = [clean_line(line.rstrip("\n").split("\t")) for line in f]
#     if len(lines) < 2:
#         print(f"⚠️ Not enough lines in {events_path}")
#         return {}
#     filenames_line = lines[0]
#     # Find the first numeric data row (assumed to be after header rows)
#     data_line = None
#     for row in lines[2:]:
#         if row and row[0].isdigit():
#             data_line = row
#             break
#     if not data_line:
#         print(f"⚠️ Could not find numeric data in {events_path}")
#         return {}
#     # If the first cell is a row index, skip it:
#     data_vals = data_line[1:] if data_line[0].isdigit() else data_line
#     events_dict = {}
#     use_len = min(len(filenames_line), len(data_vals))
#     col_idx = 0
#     while col_idx < use_len:
#         fn = filenames_line[col_idx].strip()
#         # For each chunk, attempt to get up to 3 values:
#         foot_str = data_vals[col_idx].strip() if col_idx < use_len else ""
#         max_er_str = data_vals[col_idx+1].strip() if (col_idx+1) < use_len else ""
#         rel_str = data_vals[col_idx+2].strip() if (col_idx+2) < use_len else ""
#         try:
#             fc = float(foot_str) if foot_str != "" else None
#         except:
#             fc = None
#         try:
#             rel = float(rel_str) if rel_str != "" else None
#         except:
#             rel = None
#         fc_fr = int(round(fc * capture_rate)) if fc is not None else None
#         rel_fr = int(round(rel * capture_rate)) if rel is not None else None
#         events_dict[fn] = {
#             "foot_contact_frame": fc_fr,
#             "release_frame": rel_fr
#         }
#         col_idx += 3
#     return events_dict
# 
# 
# ###############################################################################
# # 4) PARSE APlusData.txt => kinematics in chunks of 6 columns
# ###############################################################################
# def parse_aplus_kinematics(txt_path):
#     if not os.path.isfile(txt_path):
#         print(f"❌ APlusData file not found: {txt_path}")
#         return []
#     with open(txt_path, "r", encoding="utf-8") as f:
#         lines = [clean_line(line.rstrip("\n").split("\t")) for line in f]
#     if len(lines) < 2:
#         print(f"⚠️ Not enough lines in {txt_path}")
#         return []
#     filenames_line = lines[0]
#     varnames_line  = lines[1]
#     # Find the first numeric data row
#     data_line = None
#     for row in lines[2:]:
#         if row and row[0].isdigit():
#             data_line = row
#             break
#     if not data_line:
#         print(f"⚠️ Could not find numeric row in {txt_path}")
#         return []
# 
#     # If the first cell is a row index, skip it:
#     data_vals = data_line[1:] if data_line[0].isdigit() else data_line
# 
#     # We now have 7 columns per trial: 
#     #   Arm_Abduction@Footplant, Max_Abduction, Shoulder_Angle@Footplant,
#     #   Max_ER, Arm_Velo, Max_Torso_Rot_Velo, Torso_Angle@Footplant
#     chunk_size = 7
# 
#     use_len = min(len(filenames_line), len(varnames_line), len(data_vals))
#     n_trials = use_len // chunk_size
# 
#     results = []
#     for t in range(n_trials):
#         start = t * chunk_size
#         end = start + chunk_size
#         sub_vars = varnames_line[start:end]
#         sub_vals = data_vals[start:end]
#         if not sub_vars or not sub_vals:
#             continue
#         trial_dict = {}
#         for i in range(len(sub_vars)):
#             var_name = sub_vars[i].strip()
#             val_str  = sub_vals[i].strip()
#             try:
#                 val = float(val_str)
#             except:
#                 val = None
#             trial_dict[var_name] = val
#         
#         # Use the corresponding filename
#         fn = filenames_line[start].strip() if start < len(filenames_line) else f"Unknown_File_{t}"
#         trial_dict["filename"] = fn
# 
#         # Only append if we have a valid filename & some data
#         if fn != "" and any(trial_dict.get(k) is not None for k in trial_dict if k != "filename"):
#             results.append(trial_dict)
#     return results
# 
# def compute_score(
#     arm_velo: float,
#     torso_velo: float,
#     abd_footplant: float,
#     shoulder_fp: float,
#     max_er: float
# ) -> float:
#     """
#     Example scoring logic:
#       1) arm_velo * 0.005
#       2) torso_velo * 0.02
#       3) abduction@footplant => -1 * (abd_footplant * 2)
#       4) shoulder ER@footplant => piecewise logic
#       5) max_er => piecewise logic => +10 if in 180..210, 0 if in 211..220,
#          0 if in 179..(just below 180?), and -10 if <180 or >220
#     """
# 
#     # Safety for None => 0
#     arm_velo      = arm_velo      or 0
#     torso_velo    = torso_velo    or 0
#     abd_footplant = abd_footplant or 0
#     shoulder_fp   = shoulder_fp   or 0
#     max_er        = max_er        or 0
# 
#     score = 0.0
# 
#     # 1) Arm velocity
#     score += arm_velo * 0.005
# 
#     # 2) Torso velocity
#     score += torso_velo * 0.02
# 
#     # 3) Abduction @ footplant (negative factor)
#     score += -1.0 * (abd_footplant * 2)
# 
#     # 4) Shoulder ER @ footplant piecewise
#     if 35 <= shoulder_fp <= 75:
#         score += 30
#     elif 76 <= shoulder_fp <= 85:
#         score += 15
#     elif 86 <= shoulder_fp <= 95:
#         score += 0
#     elif 96 <= shoulder_fp <= 105:
#         score -= 10
#     elif shoulder_fp >= 106:
#         score -= 20
#     elif 25 <= shoulder_fp <= 34:
#         score += 15
#     elif 15 <= shoulder_fp <= 24:
#         score += 0
#     elif 5 <= shoulder_fp <= 14:
#         score -= 10
#     elif shoulder_fp < 5:
#         score -= 20
# 
#     # 5) Max_ER piecewise
#     er_score = 0
#     if 180 <= max_er <= 210:
#         er_score = 10
#     elif 211 <= max_er <= 220:
#         er_score = 0
#     elif max_er > 220:
#         er_score = -10
#     elif max_er < 180:
#         er_score = -10
#     
#     score += er_score
# 
#     return score
# 
# 
# ###############################################################################
# # 5) INGEST DATA INTO movement_data TABLE (INSERT OR REPLACE so DB builds over time)
# ###############################################################################
# def ingest_data(aPlusDataPath, aPlusEventsPath, db_path="actionPlus.sqlite"):
#     events_dict = parse_events_from_aPlus(aPlusEventsPath, capture_rate=300)
#     kinematics  = parse_aplus_kinematics(aPlusDataPath)
#     conn = sqlite3.connect(db_path)
#     c = conn.cursor()
# 
#     insert_sql = """
#     INSERT OR REPLACE INTO movement_data (
#       filename, participant_name, test_date, movement_type,
#       foot_contact_frame, release_frame,
# 
#       [Arm_Abduction@Footplant],
#       [Max_Abduction],
#       [Shoulder_Angle@Footplant],
#       [Max_ER],
#       [Arm_Velo],
#       [Max_Torso_Rot_Velo],
#       [Torso_Angle@Footplant],
# 
#       [Score]
#     ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
#     """
# 
#     for row in kinematics:
#         fn = row.get("filename", "").strip()
#         if not fn:
#             continue
#         fc = events_dict.get(fn, {}).get("foot_contact_frame")
#         rel= events_dict.get(fn, {}).get("release_frame")
#         p_name, p_date, m_type = parse_file_info(fn)
# 
#         # Pull the numeric fields from row
#         abd_fp    = row.get("Arm_Abduction@Footplant") or 0
#         max_abd   = row.get("Max_Abduction") or 0
#         shld_fp   = row.get("Shoulder_Angle@Footplant") or 0
#         max_er    = row.get("Max_ER") or 0
#         arm_velo  = row.get("Arm_Velo") or 0
#         torso_velo= row.get("Max_Torso_Rot_Velo") or 0
#         torso_ang = row.get("Torso_Angle@Footplant") or 0
# 
#         # Compute the score
#         score_val = compute_score(
#             arm_velo,
#             torso_velo,
#             abd_fp,
#             shld_fp,
#             max_er
#         )
# 
#         vals = [
#             fn,
#             p_name,
#             p_date,
#             m_type,
#             fc,
#             rel,
# 
#             abd_fp,
#             max_abd,
#             shld_fp,
#             max_er,
#             arm_velo,
#             torso_velo,
#             torso_ang,
# 
#             score_val  # <--- store it
#         ]
#         c.execute(insert_sql, vals)
# 
#     conn.commit()
#     conn.close()
# 
# 
# ###############################################################################
# # 6) Example main – adjust file paths as needed
# ###############################################################################
# if __name__ == "__main__":
#     DB_PATH = "actionPlus.sqlite"
#     APLUS_EVENTS_PATH = r"D:\Youth Pitch Design\Exports\aPlus_events.txt"
#     APLUS_DATA_PATH = r"D:\Youth Pitch Design\Exports\APlusData.txt"
#     init_db(DB_PATH)
#     ingest_data(APLUS_DATA_PATH, APLUS_EVENTS_PATH, DB_PATH)
#     print("✅ Done. Check movement_data in actionPlus.sqlite.")
# 
# ################################################################################
# # 7) Create report
# ################################################################################
# 
# import os, shutil
# import sqlite3
# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# from reportlab.pdfgen import canvas
# from reportlab.lib import colors
# from reportlab.lib.pagesizes import A4
# from reportlab.platypus import Table, TableStyle
# 
# ####################################################
# # CONFIG
# ####################################################
# DB_PATH = "actionPlus.sqlite"
# OUTPUT_DIR = r"D:\Youth Pitch Design\Reports\Action+"
# OUTPUT_DIR_TWO = r"G:\My Drive\Youth Pitch Reports\Reports\Action+"
# LOGO_PATH = r"C:\Users\q\PycharmProjects\Youth Pitch Design\8ctane - Faded 8 to Blue.png"
# 
# # Text files for velocities:
# AP_TORSO_V_FILE = r"D:\Youth Pitch Design\Exports\aPlus_torsoVelo.txt"
# AP_ARM_V_FILE   = r"D:\Youth Pitch Design\Exports\aPlus_armVelo.txt"
# 
# # Image paths
# ASSETS_DIR = r"C:\Users\q\PycharmProjects\Youth Pitch Design\assets"
# IMG_FRONT_FP  = os.path.join(ASSETS_DIR, "Front@FP.png")
# IMG_SAG_FP    = os.path.join(ASSETS_DIR, "sag@FP.png")
# IMG_SAG_MAXER = os.path.join(ASSETS_DIR, "sag@MaxER.png")
# IMG_SAG_REL   = os.path.join(ASSETS_DIR, "sag@Rel.png")
# 
# ####################################################
# # 1) DB / DATA HELPERS
# ####################################################
# def get_report_data():
#     conn = sqlite3.connect(DB_PATH)
#     df_last = pd.read_sql_query(
#         """
#         SELECT participant_name, test_date
#         FROM movement_data
#         ORDER BY id DESC
#         LIMIT 1
#         """,
#         conn
#     )
#     if df_last.empty:
#         conn.close()
#         raise ValueError("No data found in DB!")
# 
#     participant_name = df_last.iloc[0]["participant_name"]
#     test_date        = df_last.iloc[0]["test_date"]
# 
#     # Now also get the average Score:
#     query = f"""
#     SELECT movement_type,
#            AVG([Arm_Abduction@Footplant])     AS avg_abd,
#            AVG([Max_Abduction])               AS avg_max_abd,
#            AVG([Shoulder_Angle@Footplant])    AS avg_shoulder_fp,
#            AVG([Max_ER])                      AS avg_max_er,
#            AVG([Arm_Velo])                    AS avg_arm_velo,
#            AVG([Max_Torso_Rot_Velo])          AS avg_torso_velo,
#            AVG([Torso_Angle@Footplant])       AS avg_torso_angle,
#            AVG([Score])                       AS avg_score   -- <--- new
#     FROM movement_data
#     WHERE participant_name = '{participant_name}'
#       AND test_date        = '{test_date}'
#     GROUP BY movement_type
#     """
#     df = pd.read_sql_query(query, conn)
#     conn.close()
# 
#     # Sort by movement_type in a custom order
#     order = ["Pitch", "Shortstop", "Catchers", "Crow Hop"]
#     df["order"] = df["movement_type"].apply(lambda x: order.index(x) if x in order else 99)
#     df = df.sort_values("order").drop(columns="order")
#     return participant_name, test_date, df
# 
# def build_velocity_figure_all_frames(
#         torso_txt,
#         arm_txt,
#         foot_contact_row,
#         release_row,
#         *,
#         align_fc=False,       # << NEW
#         pre_frames=20,        # << only used when align_fc = True
#         post_frames=200       # <<
# ):
#     """
#     Plots the entire trial from frame=0 to frame=N-1 for both torso & arm velocity txt files.
#     We skip top 7 lines, then treat columns>=2 as data.
# 
#     The x-axis is frames from 0..(N−1).
#     We draw vertical gold lines at foot_contact_row-7 and release_row-7 if they're in range.
# 
#     This version sets the figure width=1600 & height=600 to make it wider and a bit taller.
#     """
#     # 1) Read TORSO (skip top 7 lines)
#     df_torso = pd.read_csv(torso_txt, header=None, sep="\t").iloc[7:,:].copy()
#     torso_data = df_torso.iloc[:, 2:].apply(pd.to_numeric, errors="coerce").abs()
# 
#     # 2) Read ARM
#     df_arm = pd.read_csv(arm_txt, header=None, sep="\t").iloc[7:,:].copy()
#     arm_data = df_arm.iloc[:, 2:].apply(pd.to_numeric, errors="coerce").abs()
# 
#     # 3) Number of total frames
#     total_torso = len(torso_data)
#     total_arm   = len(arm_data)
#     total_rows  = max(total_torso, total_arm)
# 
#     x = np.arange(total_rows)
#     
#     fc_adj  = (foot_contact_row or 0) - 7          # rows after header skip
#     rel_adj = (release_row      or 0) - 7
# 
#     # If the caller asked for alignment but we don’t have a real FC row,
#     # fall back to the full‑trial view.
#     if align_fc and fc_adj <= 0:
#         align_fc = False
# 
#     if align_fc:
#         start_idx = max(fc_adj - pre_frames, 0)
#         end_idx   = min(rel_adj + post_frames, total_rows - 1)
#         x_rel     = np.arange(start_idx, end_idx + 1) - fc_adj
#     else:
#         start_idx, end_idx = 0, total_rows - 1
#         x_rel     = np.arange(total_rows)
# 
#     # 2) build figure
#     fig = go.Figure()
#     fig.update_layout(
#         width=1600, height=600, template="plotly_dark",
#         title="Angular Velocities (aligned to FC)" if align_fc
#               else "Angular Velocities – Full Trial"
#     )
# 
#     # torso
#     for i, col in enumerate(torso_data.columns):
#         y = torso_data[col].values[start_idx:end_idx+1]
#         fig.add_trace(go.Scatter(
#             x=x_rel[:len(y)], y=y, mode="lines",
#             name="Torso" if i == 0 else None,
#             line=dict(color="#d62728"), opacity=.8,
#             showlegend=i==0
#         ))
# 
#     # arm
#     for j, col in enumerate(arm_data.columns):
#         y = arm_data[col].values[start_idx:end_idx+1]
#         fig.add_trace(go.Scatter(
#             x=x_rel[:len(y)], y=y, mode="lines",
#             name="Arm" if j == 0 else None,
#             line=dict(color="#2c99d4"), opacity=.8,
#             showlegend=j==0
#         ))
#     combined = pd.concat(
#         [torso_data.iloc[start_idx:end_idx + 1],
#          arm_data.iloc[start_idx:end_idx + 1]], axis=1
#     )
#     y_min = float(np.nanmin(combined.values)) if not combined.empty else 0
#     y_max = float(np.nanmax(combined.values)) if not combined.empty else 1
# 
#     # ---------------- reference lines ------------------------------
#     if align_fc:          # foot contact is at x = 0 in this view
#         fig.add_shape(type="line", x0=0, x1=0,
#                       y0=y_min, y1=y_max,
#                       line=dict(color="gold", dash="dot", width=2))
# 
#         if release_row and rel_adj > fc_adj:
#             rel_x = rel_adj - fc_adj
#             fig.add_shape(type="line", x0=rel_x, x1=rel_x,
#                           y0=y_min, y1=y_max,
#                           line=dict(color="gold", dash="dash", width=3))
#     else:                 # full‑trial view (FC / REL at raw frame #s)
#         if foot_contact_row:
#             fig.add_shape(type="line",
#                           x0=fc_adj, x1=fc_adj,
#                           y0=y_min, y1=y_max,
#                           line=dict(color="gold", dash="dot", width=2))
#         if release_row:
#             fig.add_shape(type="line",
#                           x0=rel_adj, x1=rel_adj,
#                           y0=y_min, y1=y_max,
#                           line=dict(color="gold", dash="dash", width=3))
# 
#     fig.update_layout(
#         legend=dict(orientation="h", y=1.02, x=0.05),
#         xaxis_title=("Frames (relative to Foot Contact)" if align_fc
#                      else "Frames (raw)"),
#         yaxis_title="Velocity (abs)")
#     return fig
# 
# 
# ####################################################
# # 2) PDF GENERATION
# ####################################################
# def generate_movement_report():
#     # --- (A) RETRIEVE DB FRAMES ---
#     conn = sqlite3.connect(DB_PATH)
#     df_frames = pd.read_sql_query(
#         """
#         SELECT foot_contact_frame, release_frame
#         FROM movement_data
#         ORDER BY id DESC
#         LIMIT 1
#         """,
#         conn
#     )
#     conn.close()
# 
#     if df_frames.empty:
#         foot_contact_row = 0
#         release_row      = 0
#     else:
#         foot_contact_row = df_frames.iloc[0]["foot_contact_frame"]
#         release_row      = df_frames.iloc[0]["release_frame"]
#         if pd.isna(foot_contact_row):
#             foot_contact_row = 0
#         if pd.isna(release_row):
#             release_row = 0
# 
#     # --- (B) GET SUMMARY TABLE DATA ---
#     participant_name, test_date, summary_df = get_report_data()
# 
#     # --- (C) BUILD & SAVE VELOCITY FIG ---
#     fig_velo = build_velocity_figure_all_frames(
#         AP_TORSO_V_FILE,
#         AP_ARM_V_FILE,
#         foot_contact_row,
#         release_row,
#         align_fc=True,      # << enable alignment
#         pre_frames=50,
#         post_frames=200
#     )
# 
#     velo_png = "angular_velocity.png"
#     fig_velo.write_image(velo_png)
# 
#     # --- (D) CREATE PDF ---
#     page_width, page_height = 2000, 3200
#     pdf_filename = os.path.join(OUTPUT_DIR, f"{participant_name} {test_date} Pitching Report.pdf")
#     c = canvas.Canvas(pdf_filename, pagesize=(page_width, page_height))
# 
#     # BLACK BG
#     c.setFillColorRGB(0,0,0)
#     c.rect(0, 0, page_width, page_height, fill=1, stroke=0)
# 
#     brand_border = colors.HexColor("#4887a8")
#     card_bg      = colors.HexColor("#1f1f1f")
#     text_color   = colors.HexColor("#ffffff")
# 
#     # --- HEADER ---
#     c.setFont("Helvetica-BoldOblique", 50)
#     c.setFillColor(text_color)
#     c.drawString(30, page_height - 60, "Movement Analysis Dashboard")
# 
#     c.setFont("Helvetica-Oblique", 34)
#     c.drawString(30, page_height - 120, f"Athlete: {participant_name}")
#     c.drawString(30, page_height - 170, f"Date: {test_date}")
# 
#     c.setStrokeColor(brand_border)
#     c.setLineWidth(3)
#     c.line(20, page_height - 185, page_width - 20, page_height - 185)
# 
#     # LOGO
#     if os.path.exists(LOGO_PATH):
#         c.drawImage(
#             LOGO_PATH,
#             page_width - 500,
#             page_height - 180,
#             width=398.06,
#             height=160.03,
#             preserveAspectRatio=True,
#             mask='auto'
#         )
# 
#     # --- TABLE OF AVERAGES ---
#     table_df = summary_df.copy()
#     # If your summary_df has a column 'avg_score' that you do NOT want in the table,
#     # you can drop it. Or just rename everything except that column:
#     table_df = table_df.rename(columns={
#         "avg_abd":         "Abd @ FP",
#         "avg_max_abd":     "Max Abd",
#         "avg_shoulder_fp": "Arm Timing",
#         "avg_max_er":      "Max ER",
#         "avg_arm_velo":    "Arm Velo",
#         "avg_torso_velo":  "Torso Velo",
#         "avg_torso_angle": "Torso Ang@FP"
#         # Note: if "avg_score" is in df, we won't rename it, or we can drop it:
#         # "avg_score": "Score"  # or just not rename if we don't want it in the table
#     })
# 
#     table_df = table_df.set_index("movement_type")
#     desired_order = [mt for mt in ["Pitch", "Shortstop", "Catchers", "Crow Hop"] if mt in table_df.index]
#     table_df = table_df.loc[desired_order]
# 
#     # Build list-of-lists for the table
#     header_row = ["Movement Type"] + list(table_df.columns)
#     table_data = [header_row]
#     for mt, row in table_df.iterrows():
#         row_vals = [mt]
#         for col in table_df.columns:
#             val = row[col]
#             row_vals.append(f"{val:.1f}" if pd.notna(val) else "")
#         table_data.append(row_vals)
# 
#     # Table coords & sizes
#     table_card_x = 20
#     table_card_y = page_height - 520
#     table_card_w = page_width - 480
#     table_card_h = 300
# 
#     # Draw the "card" for the table
#     c.setFillColor(card_bg)
#     c.setStrokeColor(brand_border)
#     c.roundRect(table_card_x, table_card_y, table_card_w, table_card_h, 10, fill=1)
# 
#     # Table title
#     c.setFillColor(text_color)
#     c.setFont("Helvetica-BoldOblique", 34)
#     c.drawString(table_card_x + 15, table_card_y + table_card_h - 40, "Movement Averages")
# 
#     # Build the actual table using ReportLab's Table
#     from reportlab.platypus import Table, TableStyle
#     t = Table(table_data, colWidths=[280] + [150]*(len(header_row)-1), rowHeights=48)
#     t.setStyle(TableStyle([
#         ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#333333")),
#         ('TEXTCOLOR',  (0, 0), (-1, 0), text_color),
#         ('FONTNAME',   (0, 0), (-1, 0), "Helvetica-Bold"),
#         ('FONTSIZE',   (0, 0), (-1, 0), 20),
#         ('ALIGN',      (0, 0), (-1, -1), 'CENTER'),
#         ('VALIGN',     (0, 0), (-1, -1), 'MIDDLE'),
#         ('GRID',       (0, 0), (-1, -1), 1, colors.HexColor("#444444")),
#         ('BACKGROUND', (0, 1), (-1, -1), colors.black),
#         ('TEXTCOLOR',  (0, 1), (-1, -1), text_color),
#         ('FONTNAME',   (0, 1), (-1, -1), "Helvetica"),
#         ('FONTSIZE',   (0, 1), (-1, -1), 26),
#     ]))
#     tw, th = t.wrapOn(c, table_card_w-20, table_card_h-20)
#     t.drawOn(c, table_card_x+25, table_card_y + table_card_h - 60 - th)
# 
#     ############################################################################
#     # (1) Compute the final "stability_score" from summary_df["avg_score"]
#     ############################################################################
#     if "avg_score" in summary_df.columns and not summary_df.empty:
#         stability_score = summary_df["avg_score"].mean()
#     else:
#         stability_score = 0.0
# 
#     ############################################################################
#     # (2) Draw Score Box to the RIGHT of the table
#     # So let's place it relative to table_card_x, table_card_y, etc.
#     ############################################################################
#     score_w = 340
#     score_h = 300
#     # place it 20 px to the right, and top-aligned
#     score_x = table_card_x + table_card_w + 20
#     score_y = table_card_y + (table_card_h - score_h)
# 
#     c.setFillColor(card_bg)
#     c.setStrokeColor(brand_border)
#     c.roundRect(score_x, score_y, score_w, score_h, 10, fill=1)
# 
#     c.setFillColor(text_color)
# 
#     c.setFont("Helvetica-Oblique", 24)
#     c.drawString(score_x + 10, score_y + 20, "Higher = Better")
# 
#     c.setFont("Helvetica-BoldOblique", 40)
#     c.drawString(score_x + 10, score_y + score_h - 40, "Kinematic Score")
# 
#     c.setFont("Helvetica-BoldOblique", 90)
#     c.setFillColor(colors.HexColor("#32CD32"))  # lime color
#     c.drawString(score_x + 40, score_y + 100, f"{stability_score:.1f}")
# 
#     ##########################################################
#     # ANGULAR VELOCITIES TEXT + GRAPH IN ONE BOX
#     ##########################################################
#     def draw_wrapped_text(canvas_obj, text, x, y, max_width, font_name="Helvetica", font_size=24, leading=5):
#         words = text.split()
#         lines = []
#         current_line = ""
#         for word in words:
#             test_line = f"{current_line} {word}".strip()
#             if canvas_obj.stringWidth(test_line, font_name, font_size) <= max_width:
#                 current_line = test_line
#             else:
#                 lines.append(current_line)
#                 current_line = word
#         if current_line:
#             lines.append(current_line)
#         
#         for i, line in enumerate(lines):
#             canvas_obj.drawString(x, y - i*(font_size + leading), line)
# 
#     # We place the velocity box below the table, as usual
#     top_text_graph_card_w = page_width - 100
#     top_text_graph_card_h = 800
#     top_text_graph_card_x = 50
#     top_text_graph_card_y = table_card_y - top_text_graph_card_h - 30  # same small gap
# 
#     c.setFillColor(card_bg)
#     c.setStrokeColor(brand_border)
#     c.roundRect(top_text_graph_card_x, top_text_graph_card_y,
#                 top_text_graph_card_w, top_text_graph_card_h,
#                 10, fill=1)
# 
#     c.setFillColor(text_color)
#     # Title
#     c.setFont("Helvetica-Bold", 36)
#     c.drawString(top_text_graph_card_x + 30, top_text_graph_card_y + top_text_graph_card_h - 50, 
#                  "Angular Velocities")
# 
#     # Body text
#     arm_text = (
#         "Angular velocities are how fast the designated segment rotates around the proximal segment "
#         "throughout the pitching motion. The kinematic sequence refers to sequential velocity peaks "
#         "in the pelvis, torso, upper arm, forearm, and hand. Higher velocities in the trunk and arm "
#         "have been linked to increased performance.\n\n"
#         "This chart displays the entire trial's data (frame 0 through the final frame), with vertical "
#         "gold lines at the foot contact and release frames if they exist."
#     )
# 
#     c.setFont("Helvetica", 24)
#     text_left  = top_text_graph_card_x + 30
#     text_top   = top_text_graph_card_y + top_text_graph_card_h - 100
#     max_width  = top_text_graph_card_w - 30
#     draw_wrapped_text(c, arm_text, text_left, text_top, max_width, font_size=24)
# 
#     graph_img_y = top_text_graph_card_y + 50
#     graph_img_h = 550
#     graph_side_margin = 10
#     graph_img_w = top_text_graph_card_w - (graph_side_margin * 2)
# 
#     c.drawImage(
#         velo_png,
#         text_left, 
#         graph_img_y,  
#         width=graph_img_w,
#         height=graph_img_h,
#         preserveAspectRatio=True,
#         mask='auto'
#     )
# 
#     # Move the next sections up...
#     current_y = top_text_graph_card_y - 30
# 
#     def draw_text_image_block(title_str, body_str, image_paths=None, box_height=700):
#         nonlocal current_y
#         card_w = page_width - 100
#         card_h = box_height
#         card_x = 50
#         card_y = current_y - card_h
# 
#         c.setFillColor(card_bg)
#         c.setStrokeColor(brand_border)
#         c.roundRect(card_x, card_y, card_w, card_h, 10, fill=1)
# 
#         c.setFillColor(text_color)
#         c.setFont("Helvetica-Bold", 36)
#         c.drawString(card_x + 30, card_y + card_h - 50, title_str)
# 
#         c.setFont("Helvetica", 24)
#         text_left = card_x + 30
#         text_top  = card_y + card_h - 100
#         max_text_width = card_w - 60
#         draw_wrapped_text(c, body_str, text_left, text_top, max_width=max_text_width, font_size=24)
# 
#         # Move images further down
#         img_y_offset = 20  
#         img_y = card_y + img_y_offset
#         img_w = 650
#         img_h = 500
#         if image_paths:
#             if len(image_paths) == 1:
#                 # center
#                 img_x = card_x + (card_w - img_w)/2
#                 c.drawImage(image_paths[0], img_x, img_y,
#                             width=img_w, height=img_h,
#                             preserveAspectRatio=True, mask='auto')
#             elif len(image_paths) == 2:
#                 spacing = (card_w - 2*img_w) / 3
#                 left_img_x  = card_x + spacing
#                 right_img_x = card_x + spacing*2 + img_w
#                 c.drawImage(image_paths[0], left_img_x, img_y,
#                             width=img_w, height=img_h,
#                             preserveAspectRatio=True, mask='auto')
#                 c.drawImage(image_paths[1], right_img_x, img_y,
#                             width=img_w, height=img_h,
#                             preserveAspectRatio=True, mask='auto')
# 
#         current_y = card_y - 60
#         return current_y
# 
#     # Horizontal Abduction
#     ha_title = "Horizontal Abduction"
#     ha_text  = (
#         "Horizontal abduction is how far behind the body the arm/elbow gets during the pitching motion. "
#         "Commonly referred to as the 'loading' of the arm, horizontal abduction has been linked to both velocity "
#         "and arm health.\n\n"
#         "Front@FP.png      Sag@FP.png"
#     )
#     draw_text_image_block(ha_title, ha_text, [IMG_FRONT_FP, IMG_SAG_FP], box_height=720)
# 
#     # Shoulder External Rotation
#     ser_title = "Shoulder External Rotation"
#     ser_text  = (
#         "Shoulder external rotation is measured at both footplant and as a max value during the pitching motion. "
#         "Shoulder external rotation at footplant is often referred to as 'arm timing.' An on-time arm is between "
#         "33 and 77 degrees. Anything lower than 33 is deemed late; above 77 is deemed early.\n\n"
#         "Max External rotation (often called layback) is how much the arm externally rotates during "
#         "the pitching motion. A higher max ER has been linked to both arm health and velocity.\n\n"
#         "Sag@MaxER.png      sag@Rel.png"
#     )
#     draw_text_image_block(ser_title, ser_text, [IMG_SAG_MAXER, IMG_SAG_REL], box_height=720)
# 
#     c.showPage()
#     c.save()
#     print(f"✅ PDF saved to: {pdf_filename}")
# 
#     for extra_dir in (OUTPUT_DIR_TWO,):
#         os.makedirs(extra_dir, exist_ok=True)
#         shutil.copy2(pdf_filename, os.path.join(extra_dir, os.path.basename(pdf_filename)))
#     
# if __name__ == "__main__":
#     generate_movement_report()
# 


In [ ]:
# import os
# import tkinter as tk
# from tkinter import filedialog
# 
# import numpy as np
# import ezc3d
# import sqlite3
# import pandas as pd
# import dash
# from dash import html, dcc
# from dash.dependencies import Input, Output
# import dash_bootstrap_components as dbc
# import plotly.graph_objects as go
# 
# # ===================== Discontinuity Resolution ===================== #
# def resolve_angle_discontinuities(angle_array, threshold=180.0):
#     """
#     Resolves angle discontinuities in degrees.
#     If the difference between consecutive frames is > 'threshold',
#     shift subsequent angles by ±360 degrees to keep them continuous.
#     """
#     angles = angle_array.copy()
#     for i in range(1, len(angles)):
#         diff = angles[i] - angles[i-1]
#         if diff > threshold:
#             angles[i:] -= 360.0
#         elif diff < -threshold:
#             angles[i:] += 360.0
#     return angles
# 
# # ===================== SQL Setup ===================== #
# DB_PATH = "pitch_analysis.sqlite"
# 
# conn = sqlite3.connect(DB_PATH)
# cursor = conn.cursor()
# 
# # Table for saving kinematic results
# cursor.execute("""
# CREATE TABLE IF NOT EXISTS pitch_kinematics (
#   id INTEGER PRIMARY KEY AUTOINCREMENT,
#   file_name TEXT,
#   participant_name TEXT,
#   pitch_date TEXT,
#   trial_type TEXT,
#   foot_contact_frame INTEGER,
#   ball_release_frame INTEGER,
#   peak_torso_vel REAL,
#   peak_humerus_vel REAL,
#   peak_hand_vel REAL,
#   max_abduction REAL,
#   abd_foot_contact REAL,
#   er_foot_contact REAL,
#   max_er REAL
# )
# """)
# conn.commit()
# conn.close()
# 
# # ===================== Tkinter Folder Selection ===================== #
# root = tk.Tk()
# root.withdraw()
# selected_folder = filedialog.askdirectory(title="Select Folder Containing C3D Files")
# if not selected_folder:
#     raise ValueError("No folder was selected.")
# 
# # Gather all .c3d in folder
# c3d_files = [os.path.join(selected_folder, f) for f in os.listdir(selected_folder)
#              if f.lower().endswith(".c3d")]
# 
# if not c3d_files:
#     raise ValueError("No .c3d files found in the selected folder!")
# 
# # Identify static trial(s)
# static_files = [f for f in c3d_files if "static" in os.path.basename(f).lower()]
# if not static_files:
#     print("No static trial found. Using raw angles without normalization.")
# else:
#     print("Found static files:")
#     for sf in static_files:
#         print("  ", os.path.basename(sf))
# 
# # ===================== Functions to Load & Process C3D ===================== #
# def load_c3d_file(c3d_path):
#     """ Load C3D file and extract marker data. """
#     c3d_obj = ezc3d.c3d(c3d_path)
#     points = c3d_obj["data"]["points"]  # shape (4, #markers, #frames)
#     labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
#     frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
#     n_frames = points.shape[2]
# 
#     # Debug: Print marker names to confirm correct mapping
#     print(f"\nAvailable Markers in {os.path.basename(c3d_path)}:")
#     print(labels)
# 
#     # Create dictionary with marker names
#     markers = {lab.upper(): points[:3, i, :].T for i, lab in enumerate(labels)}
# 
#     # Detect hand (Left or Right)
#     hand = "Right" if any("RIGHT_" in lab.upper() for lab in labels) else "Left"
# 
#     return c3d_obj, markers, frame_rate, n_frames, hand
# 
# 
# def detect_foot_contact_and_ball_release(events, markers, frame_rate):
#     """
#     Return foot_contact_frame, ball_release_frame
#     If events dictionary has them, use them. Otherwise, do a heuristic with markers.
#     """
#     fc_frame = None
#     br_frame = None
#     # If events exist, attempt to find foot contact and ball release by name
#     for lab, val in events.items():
#         if "foot" in lab and "contact" in lab:
#             fc_frame = val
#         if "ball" in lab and "release" in lab:
#             br_frame = val
#     # If not found, do a fallback method:
#     # e.g., foot contact: a foot marker's vertical velocity crosses 0
#     if fc_frame is None:
#         possible_foot_markers = [m for m in markers if "HEEL" in m or "TOE" in m or "FOOT" in m]
#         if possible_foot_markers:
#             fm = possible_foot_markers[0]
#             Z = markers[fm][:, 2]
#             vz = np.gradient(Z) * frame_rate
#             for i in range(1, len(Z)):
#                 if vz[i-1] < 0 and vz[i] >= 0:
#                     fc_frame = i
#                     break
#     if fc_frame is None:
#         fc_frame = 0  # default to start
# 
#     # ball release: if a 'ball' marker is present, find separation from hand
#     if br_frame is None:
#         ball_marker = [m for m in markers if "BALL" in m]
#         hand_marker = [m for m in markers if "WRIST" in m or "HAND" in m]
#         if ball_marker and hand_marker:
#             b = ball_marker[0]
#             h = hand_marker[0]
#             dist = np.linalg.norm(markers[b] - markers[h], axis=1)
#             for i in range(1, len(dist)):
#                 if dist[i] > 0.03:  # 3 cm threshold
#                     br_frame = i
#                     break
#     if br_frame is None or br_frame < fc_frame:
#         br_frame = fc_frame + int(frame_rate * 0.5)  # default 0.5s after FC
# 
#     return fc_frame, br_frame
# 
# 
# def compute_torso_humerus_hand_kinematics(markers, frame_rate, static_offsets=None):
#     """
#     Computes joint angles and segment velocities using the correct marker names.
#     """
# 
#     print("\nProcessing markers for joint angle calculations:")
#     print(list(markers.keys()))  # Debugging: Print available markers
# 
#     # Correct marker names based on provided list
#     RSHO = markers.get("R_Shoulder")
#     LSHO = markers.get("L_Shoulder")
#     RELB_Lat = markers.get("Right_Lateral_Elbow")
#     RELB_Med = markers.get("Right_Medial_Elbow")
#     RWR_Rad = markers.get("Right_Wrist_Radius")
#     RWR_Ulna = markers.get("Right_Wrist_Ulna")
#     HAND = markers.get("Right_Hand")
# 
#     if not all([RSHO, LSHO, RELB_Lat, RELB_Med, RWR_Rad, RWR_Ulna, HAND]):
#         print("Missing required markers! Skipping this trial.")
#         return None
# 
#     # Compute elbow center (midpoint of lateral and medial elbow)
#     RELB = (RELB_Lat + RELB_Med) / 2.0
# 
#     # Compute wrist center (midpoint of radius and ulna)
#     RWR = (RWR_Rad + RWR_Ulna) / 2.0
# 
#     # Compute segment vectors
#     torso_vec = RSHO - LSHO  # Shoulder-to-shoulder axis
#     humerus_vec = RELB - RSHO  # Shoulder to elbow
#     forearm_vec = RWR - RELB  # Elbow to wrist
# 
#     # Shoulder Abduction (Angle between humerus and vertical plane)
#     shoulder_abduction = np.degrees(np.arccos(np.clip(
#         np.dot(humerus_vec / np.linalg.norm(humerus_vec, axis=1, keepdims=True),
#                np.array([0, 0, 1])),
#         -1.0, 1.0)))
# 
#     # External Rotation (Angle between forearm and humerus in the transverse plane)
#     plane_normal = np.cross(humerus_vec, np.array([0, 0, 1]))  # Plane perpendicular to humerus
#     hand_proj = forearm_vec - np.dot(forearm_vec, plane_normal)[:, np.newaxis] * plane_normal
# 
#     shoulder_external_rot = np.degrees(np.arccos(np.clip(
#         np.dot(humerus_vec / np.linalg.norm(humerus_vec, axis=1, keepdims=True),
#                hand_proj / np.linalg.norm(hand_proj, axis=1, keepdims=True)),
#         -1.0, 1.0)))
# 
#     # Compute Angular Velocities
#     torso_ang_vel = np.gradient(np.degrees(np.arccos(np.clip(
#         np.dot(torso_vec[:-1], torso_vec[1:].T), -1.0, 1.0)))) * frame_rate
#     humerus_ang_vel = np.gradient(shoulder_abduction) * frame_rate
#     hand_ang_vel = np.gradient(shoulder_external_rot) * frame_rate
# 
#     return {
#         "shoulder_abduction": shoulder_abduction,
#         "shoulder_external_rot": shoulder_external_rot,
#         "torso_vel": torso_ang_vel,
#         "humerus_vel": humerus_ang_vel,
#         "hand_vel": hand_ang_vel
#     }
# 
# def compute_static_offsets(static_file):
#     _, markers, frame_rate, _, _ = load_c3d_file(static_file)
#     kin = compute_torso_humerus_hand_kinematics(markers, frame_rate)
# 
#     if kin is None:
#         print(f"WARNING: Could not compute static offsets from {static_file}")
#         return {}
# 
#     mean_abd = np.mean(kin["shoulder_abduction"])
#     mean_er = np.mean(kin["shoulder_external_rot"])
# 
#     print(f"Static Offsets -> Abduction: {mean_abd:.2f}, External Rotation: {mean_er:.2f}")
# 
#     return {"abd": mean_abd, "er": mean_er}
# 
# 
# 
# # ===================== Compute Static Offsets if available ===================== #
# static_offsets = {}
# if static_files:
#     # Use the first static file found
#     static_offsets = compute_static_offsets(static_files[0])
#     print("Static Offsets:", static_offsets)
# 
# # ===================== MAIN PROCESSING LOOP ===================== #
# conn = sqlite3.connect(DB_PATH)
# cursor = conn.cursor()
# 
# for file_path in c3d_files:
#     base_name = os.path.basename(file_path)
#     # skip static file in main loop
#     if "static" in base_name.lower():
#         continue
# 
#     print(f"Processing {base_name}")
#     try:
#         c3d_obj, markers, frame_rate, n_frames, events = load_c3d_file(file_path)
#     except Exception as e:
#         print(f"Error loading {base_name}: {e}")
#         continue
# 
#     # Heuristics or events to find foot contact & ball release
#     fc_frame, br_frame = detect_foot_contact_and_ball_release(events, markers, frame_rate)
# 
#     # compute angles & velocities
#     kin = compute_torso_humerus_hand_kinematics(markers, frame_rate, static_offsets)
#     if kin is None:
#         print(f"Skipping {base_name}; could not compute angles.")
#         continue
# 
#     # Extract metrics from FC..BR
#     fc_frame = max(fc_frame, 0)
#     br_frame = min(br_frame, n_frames-1)
#     if br_frame <= fc_frame:
#         print(f"Skipping {base_name}: ball release <= foot contact.")
#         continue
# 
#     # Peak velocities in the window
#     torso_in_window = kin["torso_vel"][fc_frame:br_frame+1]
#     hum_in_window   = kin["humerus_vel"][fc_frame:br_frame+1]
#     hand_in_window  = kin["hand_vel"][fc_frame:br_frame+1]
# 
#     peak_torso = np.max(torso_in_window)
#     peak_hum   = np.max(hum_in_window)
#     peak_hand  = np.max(hand_in_window)
# 
#     # Max abduction overall
#     abd_in_window = kin["shoulder_abduction"][fc_frame:br_frame+1]
#     max_abd = np.max(abd_in_window)
#     abd_fc = kin["shoulder_abduction"][fc_frame]
# 
#     # Shoulder external rotation at foot contact & max
#     er_fc = kin["shoulder_external_rot"][fc_frame]
#     er_in_window = kin["shoulder_external_rot"][fc_frame:br_frame+1]
#     max_er = np.max(er_in_window)
# 
#     # Insert into DB
#     # We can parse participant_name, pitch_date, trial_type from folder structure if desired
#     # For example, folder structure: root\\Participant01\\2025-02-21\\Fastball1.c3d
#     # We'll do a simple parse here:
#     folder_parts = os.path.normpath(file_path).split(os.sep)
#     # e.g. [ ..., 'Participant01_123', '2025-02-21_', 'Fastball1.c3d']
#     participant_name = None
#     pitch_date = None
#     trial_type = "Unknown"
# 
#     if len(folder_parts) >= 3:
#         participant_name = folder_parts[-3]  # e.g. 'Participant01_123'
#         pitch_date = folder_parts[-2].replace("_", "")
#         # trial_type might be derived from the filename
#         trial_type = os.path.splitext(folder_parts[-1])[0]
#     
#     if not participant_name:
#         participant_name = "Unknown"
#     if not pitch_date:
#         pitch_date = "Unknown"
# 
#     insert_sql = """
#     INSERT INTO pitch_kinematics (
#         file_name, participant_name, pitch_date, trial_type,
#         foot_contact_frame, ball_release_frame,
#         peak_torso_vel, peak_humerus_vel, peak_hand_vel,
#         max_abduction, abd_foot_contact, er_foot_contact, max_er
#     ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
#     """
# 
#     data_tuple = (
#         base_name,
#         participant_name,
#         pitch_date,
#         trial_type,
#         fc_frame,
#         br_frame,
#         float(peak_torso),
#         float(peak_hum),
#         float(peak_hand),
#         float(max_abd),
#         float(abd_fc),
#         float(er_fc),
#         float(max_er)
#     )
#     cursor.execute(insert_sql, data_tuple)
#     conn.commit()
# 
# conn.close()
# 
# print("Done processing all trials. Data stored in pitch_kinematics table.")
# 
# 
# # ===================== BUILD DASH APP ===================== #
# # We'll load external_stylesheets with your custom CSS
# # Make sure you have the 'assets/custom.css' in your Dash project folder.
# # Then you can do:
# external_stylesheets = [dbc.themes.BOOTSTRAP, "assets/custom.css"]
# 
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
# 
# app.title = "Youth Pitch Design Kinematics"
# 
# # Layout
# app.layout = dbc.Container([
#     dbc.Row([
#         dbc.Col(html.H1("Pitch Kinematics", className="text-center text-primary mb-4"),
#                 width=12)
#     ]),
#     dbc.Row([
#         # We'll add drop-downs to select participant, date, pitch type, etc.
#         # Then display a graph of velocities or angles from the DB.
#         dbc.Col([
#             html.Label("Participant"),
#             dcc.Dropdown(id="participant-dropdown",
#                          style={"margin-bottom": "20px"}),
#             html.Label("Date"),
#             dcc.Dropdown(id="date-dropdown",
#                          style={"margin-bottom": "20px"}),
#             html.Label("Trial Type"),
#             dcc.Dropdown(id="trial-dropdown",
#                          style={"margin-bottom": "20px"}),
#             dbc.Button("Refresh Data", id="refresh-btn", color="primary", className="w-100")
#         ], width=3),
#         dbc.Col([
#             dcc.Graph(id="velocity-graph")
#         ], width=9)
#     ])
# ], fluid=True)
# 
# @app.callback(
#     Output("participant-dropdown", "options"),
#     Output("participant-dropdown", "value"),
#     Input("refresh-btn", "n_clicks")
# )
# def update_participants(_):
#     # Load participants from DB
#     conn = sqlite3.connect(DB_PATH)
#     df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_kinematics", conn)
#     conn.close()
#     participants = df["participant_name"].unique().tolist()
#     options = [{"label": p, "value": p} for p in participants]
#     val = participants[0] if participants else None
#     return options, val
# 
# @app.callback(
#     Output("date-dropdown", "options"),
#     Output("date-dropdown", "value"),
#     Input("participant-dropdown", "value")
# )
# def update_dates(participant):
#     if not participant:
#         return [], None
#     conn = sqlite3.connect(DB_PATH)
#     query = f"SELECT DISTINCT pitch_date FROM pitch_kinematics WHERE participant_name='{participant}'"
#     df = pd.read_sql_query(query, conn)
#     conn.close()
#     dates = df["pitch_date"].unique().tolist()
#     options = [{"label": d, "value": d} for d in dates]
#     return options, (dates[0] if dates else None)
# 
# @app.callback(
#     Output("trial-dropdown", "options"),
#     Output("trial-dropdown", "value"),
#     Input("participant-dropdown", "value"),
#     Input("date-dropdown", "value")
# )
# def update_trial_types(participant, date):
#     if not participant or not date:
#         return [], None
#     conn = sqlite3.connect(DB_PATH)
#     query = f"""
#     SELECT DISTINCT trial_type FROM pitch_kinematics
#     WHERE participant_name='{participant}' AND pitch_date='{date}'
#     """
#     df = pd.read_sql_query(query, conn)
#     conn.close()
#     trials = df["trial_type"].unique().tolist()
#     options = [{"label": t, "value": t} for t in trials]
#     return options, (trials[0] if trials else None)
# 
# @app.callback(
#     Output("velocity-graph", "figure"),
#     Input("participant-dropdown", "value"),
#     Input("date-dropdown", "value"),
#     Input("trial-dropdown", "value")
# )
# def update_graph(participant, date, trial):
#     if not participant or not date or not trial:
#         return go.Figure()
# 
#     conn = sqlite3.connect(DB_PATH)
#     query = f"""
#     SELECT file_name, peak_torso_vel, peak_humerus_vel, peak_hand_vel
#     FROM pitch_kinematics
#     WHERE participant_name='{participant}' AND pitch_date='{date}' AND trial_type='{trial}'
#     """
#     df = pd.read_sql_query(query, conn)
#     conn.close()
# 
#     fig = go.Figure()
#     fig.add_trace(go.Bar(
#         x=df["file_name"],
#         y=df["peak_torso_vel"],
#         name="Peak Torso Vel"
#     ))
#     fig.add_trace(go.Bar(
#         x=df["file_name"],
#         y=df["peak_humerus_vel"],
#         name="Peak Humerus Vel"
#     ))
#     fig.add_trace(go.Bar(
#         x=df["file_name"],
#         y=df["peak_hand_vel"],
#         name="Peak Hand Vel"
#     ))
# 
#     fig.update_layout(barmode='group',
#                       title="Peak Angular Velocities",
#                       xaxis_title="Trial",
#                       yaxis_title="Velocity (deg/s)")
#     return fig
# 
# if __name__ == "__main__":
#     app.run_server(debug=True)
